In [ ]:
import xarray as xr
import cfgrib
%pylab inline
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')
from wrf import smooth2d

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.
G = 9.81

def get_z(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'gh'}})
    var = file.get('gh')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_u(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'u'}})
    var = file.get('u')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_v(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'v'}})
    var = file.get('v')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_t(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa', 'shortName': 't'}})
    var = file.get('t')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def calc_dx_dy(longitude,latitude):
    ''' This definition calculates the distance between grid points that are in
        a latitude/longitude format.
        
        Equations from:
        http://andrew.hedges.name/experiments/haversine/

        dy should be close to 55600 m
        dx at pole should be 0 m
        dx at equator should be close to 55600 m
        
        Accepts, 1D arrays for latitude and longitude
        
        Returns: dx, dy; 2D arrays of distances between grid points 
                                    in the x and y direction in meters 
    '''
    dlat = np.abs(latitude[1]-latitude[0])*np.pi/180
    dy = 2*(np.arctan2(np.sqrt((np.sin(dlat/2))**2),np.sqrt(1-(np.sin(dlat/2))**2)))*6371000
    dy = np.ones((latitude.shape[0],longitude.shape[0]))*dy

    dx = np.empty((latitude.shape))
    dlon = np.abs(longitude[1] - longitude[0])*np.pi/180
    for i in range(latitude.shape[0]):
        a = (np.cos(latitude[i]*np.pi/180)*np.cos(latitude[i]*np.pi/180)*np.sin(dlon/2))**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a) )
        dx[i] = c * 6371000
    dx = np.repeat(dx[:,np.newaxis],longitude.shape,axis=1)
    return dx, dy


In [ ]:
min_lat1 = 35
max_lat1 = 45
min_lon1 = -80
max_lon1 = -65

z = get_z('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

u = get_u('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

v = get_v('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

t = get_t('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

dx, dy = calc_dx_dy(np.asarray(z['longitude']),np.asarray(z['latitude']))

zl = np.asarray(z['gh'])
tl = np.asarray(t['t'])*(P0/500)**(RD/CP)
vl = np.asarray(v['v'])
ul = np.asarray(u['u'])

### Gradients
du_dx = np.gradient(ul,axis=1)/dx
dv_dx = np.gradient(vl,axis=1)/dx
dt_dx = np.gradient(tl,axis=1)/dx

du_dy = np.gradient(ul,axis=0)/dy
dv_dy = np.gradient(vl,axis=0)/dy
dt_dy = np.gradient(tl,axis=0)/dy

qx = (-G/TR)*(du_dx*dt_dx + dv_dx*dt_dy)
qy = (-G/TR)*(du_dy*dt_dx + dv_dy*dt_dy)
divq = -2*(np.gradient(qx,axis=1)/dx + np.gradient(qy,axis=0)/dy)

z_f = get_z('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

u_f = get_u('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

v_f = get_v('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

t_f = get_t('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
            min_lat1, max_lat1, min_lon1, max_lon1)

zl_f = np.asarray(z_f['gh'])
tl_f = np.asarray(t_f['t'])*(P0/500)**(RD/CP)
vl_f = np.asarray(v_f['v'])
ul_f = np.asarray(u_f['u'])

### Gradients
du_dx_f = np.gradient(ul_f,axis=1)/dx
dv_dx_f = np.gradient(vl_f,axis=1)/dx
dt_dx_f = np.gradient(tl_f,axis=1)/dx

du_dy_f = np.gradient(ul_f,axis=0)/dy
dv_dy_f = np.gradient(vl_f,axis=0)/dy
dt_dy_f = np.gradient(tl_f,axis=0)/dy

qx_f = (-G/TR)*(du_dx_f*dt_dx_f + dv_dx_f*dt_dy_f)
qy_f = (-G/TR)*(du_dy_f*dt_dx_f + dv_dy_f*dt_dy_f)
divq_f = -2*(np.gradient(qx_f,axis=1)/dx + np.gradient(qy_f,axis=0)/dy)


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/2

fig, axd = plt.subplot_mosaic([['left','right'],['cbar','cbar']],
                              constrained_layout=True, figsize=(8.4,3.8), dpi=175, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':crs.PlateCarree()},
                                              'right':{'projection':crs.PlateCarree()}})

##############
### LEFT
#############

cs1 = axd['left'].contour(z['longitude'], z['latitude'], smooth2d(zl,4)/10.,
                  levels=np.arange(0,1500,5),
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,5),fontsize=6,colors='k')

cs2 = axd['left'].contour(t['longitude'], t['latitude'], tl,
                  colors='b', transform=crs.PlateCarree(),linestyles='dashed')
# axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(-50,0,2),fontsize=6,
#                   colors='b')

im1 = axd['left'].contourf(t['longitude'], t['latitude'], divq*1e14,extend='both',levels=zlvls,
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

axd['left'].set_extent((min_lon1,max_lon1,min_lat1,max_lat1))

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
# gls.xlocator = mticker.FixedLocator([ -80, -76, -72])
# gls.ylocator = mticker.FixedLocator([38, 40, 42, 44])

axd['left'].set_title('GFS anl 0000 UTC 16 Nov 2018')

##############
### RIGHT
#############

cs3 = axd['right'].contour(z_f['longitude'], z_f['latitude'], zl_f/10.,
                  levels=np.arange(0,1500,5),
                  colors='k', transform=crs.PlateCarree())
axd['right'].clabel(cs3,fmt='%1.0f',inline=1,levels=np.arange(0,1500,5),fontsize=6,colors='k')

cs4 = axd['right'].contour(t_f['longitude'], t_f['latitude'], tl_f,
                  colors='b', transform=crs.PlateCarree(),linestyles='dashed')
# axd['right'].clabel(cs4,fmt='%1.0f',inline=1,levels=np.arange(-50,0,2),fontsize=6,
#                   colors='b')

im2 = axd['right'].contourf(t_f['longitude'], t_f['latitude'], divq_f*1e14,levels=zlvls,
                    extend='max',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

axd['right'].set_extent((min_lon1,max_lon1,min_lat1,max_lat1))

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=False
# gls.xlocator = mticker.FixedLocator([ -80, -76, -72])
# gls.ylocator = mticker.FixedLocator([38, 40, 42, 44])

axd['right'].set_title('GFS fcs 0000 UTC 16 Nov 2018')

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])

plt.show()
